In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skmultilearn.model_selection import iterative_train_test_split
import wandb
from wandb.keras import WandbCallback

2022-06-27 16:59:41.418393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
tf.__version__

'2.4.1'

# Data Load

In [4]:
####### 자료 경로
df = pd.read_csv('/home/lab38/Multi_proj_6/data/lettu_smallsizeimg_with_pest.csv')
df.head()

,image,grow,disease,disease-grow,area,points,original
0,/home/lab38/상추/358.jpg,1,12,12-1,NaN,NaN,NaN
1,/home/lab38/상추/235.jpg,1,12,12-1,NaN,NaN,NaN
2,/home/lab38/상추/39.jpg,0,12,12-0,NaN,NaN,NaN
3,/home/lab38/상추/345.jpg,1,12,12-1,NaN,NaN,NaN
4,/home/lab38/상추/97.jpg,1,12,12-1,NaN,NaN,NaN


In [5]:
# label encoding
disease_encoder = LabelEncoder()
disease_encoder.fit(df['disease'])
df['disease'] = disease_encoder.transform(df['disease'])
print(df['disease'].unique())

grow_encoder = LabelEncoder()
grow_encoder.fit(df['grow'])
df['grow'] = grow_encoder.transform(df['grow'])
print(df['grow'].unique())

[3 0 1 2]
[1 0]


In [6]:
shuffled_df = df.sample(frac=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['image'],
                                                   df['disease-grow'],
                                                   stratify=df['disease-grow'],
                                                   test_size=0.2)

In [8]:
train_df = df[df['image'].isin(X_train)]
test_df = df[df['image'].isin(X_test)]

In [9]:
print(train_df['disease'].unique())
print(train_df['grow'].unique())
print(test_df['disease'].unique())
print(test_df['grow'].unique())

[3 0 1 2]
[1 0]
[3 0 1 2]
[1 0]


In [11]:
test_df.to_csv('/home/lab38/Multi_proj_6/data/lettuce_valid_set.csv', index=False)

In [10]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20, # 최대 20도까지 회전
                            width_shift_range=0.1, # 최대 x 범위안에서 좌우/상하 이동
                            height_shift_range=0.1,
                            zoom_range=0.2, # 확대 축소 비율,
                            horizontal_flip=True, # 좌우반전
                            vertical_flip=True, # 상하반전
                            fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale= 1. /255.)

# Model Top Layer 학습

In [11]:
config_default = {
    'pretrain_net': 'inception',
    'epochs' : 1000,
    'batch_size': 50,
    'dropout' : 0.2,
    'learning_rate' : 1e-3,
    'activation': 'relu',
    'optimizer': 'rmsprop',
    'dense': 32,
}
wandb.init(project='lettuce',
          config=config_default)
config = wandb.config
IMAGE_SIZE = 320

train_generator = train_gen.flow_from_dataframe(train_df, 
                                           x_col='image',
                                           y_col=['disease', 'grow'],
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           class_mode='multi_output',
                                           batch_size=config.batch_size)
valid_generator = valid_gen.flow_from_dataframe(test_df,
                                           x_col='image',
                                           y_col=['disease','grow'],
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           class_mode='multi_output',
                                           batch_size=config.batch_size)

if config.pretrain_net == 'inception' :
    MODEL_IMAGE_SIZE = 299
    base_model = inception_resnet_v2.InceptionResNetV2(
        weights='imagenet',
        include_top = False,
        input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
    )
base_model.trainable = False 

# resizing model
input_data = layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
x = tf.keras.layers.experimental.preprocessing.Resizing(MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE)(input_data)
resizing = Model(inputs=input_data, outputs=x, name='resize')

# model
inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = resizing(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(config.dropout)(x)
backbone_out = layers.Dense(config.dense, activation=config.activation)(x)

disease_outputs = layers.Dense(df['disease'].nunique(), activation='softmax',
                        name = 'diease_outputs')(backbone_out)
grow_outputs = layers.Dense(1, activation='sigmoid',
                    name = 'grow_outputs')(backbone_out)

model = Model(inputs=inputs, 
              outputs=[disease_outputs, grow_outputs],
              name='lettuce') 

if config.optimizer=='rmsprop':
    optimizer = RMSprop(learning_rate=config.learning_rate)

es = EarlyStopping(monitor='loss',
                   mode='auto',
                  patience=5,
                  verbose=1)

######### 경로설정 ############
ckpt_path = './toplayer_lettuce.ckpt'
checkpointer = ModelCheckpoint(filepath=ckpt_path,
                              monitor='val_loss',
                              save_weights_only = True,
                              save_best_only= True,
                              verbose=1)
    
model.compile(loss={
                  'diease_outputs' : 'sparse_categorical_crossentropy',
                  'grow_outputs' : 'binary_crossentropy'
              },
              optimizer=optimizer,
              metrics=['accuracy'])

history = model.fit(train_generator,
                  validation_data=valid_generator,
                  verbose=1,
                  epochs=config.epochs,
                  callbacks=[es, checkpointer],
                  steps_per_epoch=len(train_df)//config.batch_size)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 4738 validated image filenames.
Found 1185 validated image filenames.


2022-06-24 09:33:28.062428: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-24 09:33:28.063480: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-24 09:33:28.106500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-24 09:33:28.107094: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-24 09:33:28.107144: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-24 09:33:28.108939: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/1000


2022-06-24 09:33:43.508320: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-24 09:33:44.572403: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


94/94 [==============================] - 129s 1s/step - loss: 1.2834 - diease_outputs_loss: 0.9505 - grow_outputs_loss: 0.3328 - diease_outputs_accuracy: 0.6119 - grow_outputs_accuracy: 0.8538 - val_loss: 0.6003 - val_diease_outputs_loss: 0.4445 - val_grow_outputs_loss: 0.1559 - val_diease_outputs_accuracy: 0.8489 - val_grow_outputs_accuracy: 0.9502

Epoch 00001: val_loss improved from inf to 0.60034, saving model to ./toplayer_lettuce.ckpt
Epoch 2/1000
94/94 [==============================] - 113s 1s/step - loss: 0.6011 - diease_outputs_loss: 0.4432 - grow_outputs_loss: 0.1580 - diease_outputs_accuracy: 0.8349 - grow_outputs_accuracy: 0.9438 - val_loss: 0.4727 - val_diease_outputs_loss: 0.3500 - val_grow_outputs_loss: 0.1227 - val_diease_outputs_accuracy: 0.8768 - val_grow_outputs_accuracy: 0.9620

Epoch 00002: val_loss improved from 0.60034 to 0.47266, saving model to ./toplayer_lettuce.ckpt
Epoch 3/1000
94/94 [==============================] - 113s 1s/step - loss: 0.5123 - diease_ou

KeyboardInterrupt: 

####################### 여기까지 ########################

# Sweep
finetuning freezing rate

In [11]:
def finetune() :
    IMAGE_SIZE = 320
    config_defaults = {
        'pretrain_net': 'inception',
        'epochs' : 1000,
        'batch_size': 20,
        'dropout' : 0.2,
        'learning_rate' : 1e-3,
        'activation': 'elu',
        'optimizer': 'rmsprop',
        'dense': 32,
        'freeze_rate' : 0.3
    }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_generator = train_gen.flow_from_dataframe(train_df, 
                                           x_col='image',
                                           y_col=['disease', 'grow'],
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           class_mode='multi_output',
                                           batch_size=config.batch_size)
    valid_generator = valid_gen.flow_from_dataframe(test_df,
                                           x_col='image',
                                           y_col=['disease','grow'],
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           class_mode='multi_output',
                                           batch_size=config.batch_size)
    
    ################### model 구성 ####################
    if config.pretrain_net == 'inception' :
        MODEL_IMAGE_SIZE = 299
        base_model = inception_resnet_v2.InceptionResNetV2(
            weights='imagenet',
            include_top = False,
            input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
        )
    base_model.trainable = False 
    
    # resizing model
    input_data = layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
    x = tf.keras.layers.experimental.preprocessing.Resizing(MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE)(input_data)
    resizing = Model(inputs=input_data, outputs=x, name='resize')
    
    
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)) 
    x = resizing(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(config.dropout)(x)
    backbone_out = layers.Dense(config.dense, activation=config.activation)(x)

    disease_outputs = layers.Dense(df['disease'].nunique(), activation='softmax',
                            name = 'diease_outputs')(backbone_out)
    grow_outputs = layers.Dense(1, activation='sigmoid',
                        name = 'grow_outputs')(backbone_out)

    model = Model(inputs=inputs, 
                  outputs=[disease_outputs, grow_outputs],
                  name='lettuce') 
    ###########################################################################3
    
    ####### top layer weight & bias loading ######
    try :
        ######### 경로 설정 ##########
        ckpt_path = './toplayer_lettuce.ckpt'
        model.load_weights=ckpt_path
    except :
        print('fail to load weights')
        
    ############### 동결 해제 ###################
    base_model.trainable=True
    fine_tune_at = int(len(base_model.layers) * config.freeze_rate)
    for layer in base_model.layers[:fine_tune_at] :
        layer.trainable = False 
        
    if config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate / 10)
        
    model.compile(loss={
                  'diease_outputs' : 'sparse_categorical_crossentropy',
                  'grow_outputs' : 'binary_crossentropy'
               },
                optimizer=optimizer,
                metrics=['accuracy'])
    
    
    es = EarlyStopping(monitor='loss',
                   mode='auto',
                  patience=5,
                  verbose=1)
    ckpt_path_ = './finetune_lettuce_{}.ckpt'.format(config.freeze_rate)
    checkpointer = ModelCheckpoint(filepath=ckpt_path,
                              monitor='val_loss',
                              save_weights_only = True,
                              save_best_only= True,
                              verbose=1)
    history_fine = model.fit(train_generator,
                              validation_data=valid_generator,
                              verbose=1,
                              epochs=config.epochs,
                              callbacks=[es, WandbCallback()],
                              steps_per_epoch=len(train_df)//config.batch_size)    

In [12]:
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'freeze_rate': {
            'values': [0, 0.2, 0.5, 0.9]
        }
    }
}

In [13]:
sweep_id = wandb.sweep(sweep_config, project='lettuce')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: b37xduwh
Sweep URL: https://wandb.ai/hkleee/lettuce/sweeps/b37xduwh


In [ ]:
wandb.agent(sweep_id, finetune)

wandb: Agent Starting Run: 4b87ikvf with config:
wandb: 	freeze_rate: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 4738 validated image filenames.
Found 1185 validated image filenames.


2022-06-24 15:26:03.505082: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-24 15:26:03.506152: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-24 15:26:03.551126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-24 15:26:03.551711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-24 15:26:03.551762: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-24 15:26:03.553685: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/1000


2022-06-24 15:26:33.964586: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-24 15:26:34.243722: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


...

wandb: Agent Starting Run: dj07wkgs with config:
wandb: 	freeze_rate: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4738 validated image filenames.
Found 1185 validated image filenames.
Epoch 1/1000
236/236 [==============================] - 204s 761ms/step - loss: 1.1697 - diease_outputs_loss: 0.8685 - grow_outputs_loss: 0.3011 - diease_outputs_accuracy: 0.6047 - grow_outputs_accuracy: 0.8905 - val_loss: 0.6914 - val_diease_outputs_loss: 0.4107 - val_grow_outputs_loss: 0.2807 - val_diease_outputs_accuracy: 0.8616 - val_grow_outputs_accuracy: 0.8633
Epoch 2/1000
236/236 [==============================] - 179s 756ms/step - loss: 0.4861 - diease_outputs_loss: 0.3264 - grow_outputs_loss: 0.1598 - diease_outputs_accuracy: 0.8880 - grow_outputs_accuracy: 0.9396 - val_loss: 0.2264 - val_diease_outputs_loss: 0.1551 - val_grow_outputs_loss: 0.0713 - val_diease_outputs_accuracy: 0.9451 - val_grow_outputs_accuracy: 0.9730
Epoch 3/1000
236/236 [==============================] - 180s 759ms/step - loss: 0.3260 - diease_outputs_loss: 0.2317 - grow_outputs_loss: 0.0943 - diease_outputs_accuracy: 0.9200 - gro

diease_outputs_accuracy,▁▆▇▇█████████████
diease_outputs_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▂▂▂
epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
grow_outputs_accuracy,▁▄▆▇▇█▇███▇███▇▇▇
grow_outputs_loss,█▄▂▂▂▂▂▂▁▁▂▁▂▁▃▃▄
loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▂▂▂
val_diease_outputs_accuracy,▃▇▁▇████▇█▇▇▆█▇▇▇
val_diease_outputs_loss,▆▂█▂▂▁▁▁▂▁▂▁▅▃▂▄▄
val_grow_outputs_accuracy,▁▇▃▇█▇▅███▇▇▇▇▇█▇
val_grow_outputs_loss,█▂▇▄▁▁▄▁▁▁▃▄▃▃▂▂▃
val_loss,▇▂█▃▂▁▂▁▁▁▂▂▄▃▂▃▄


wandb: Agent Starting Run: ow852ohm with config:
wandb: 	freeze_rate: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4738 validated image filenames.
Found 1185 validated image filenames.
Epoch 1/1000
236/236 [==============================] - 198s 742ms/step - loss: 1.0883 - diease_outputs_loss: 0.7761 - grow_outputs_loss: 0.3121 - diease_outputs_accuracy: 0.6808 - grow_outputs_accuracy: 0.8732 - val_loss: 0.2975 - val_diease_outputs_loss: 0.1996 - val_grow_outputs_loss: 0.0979 - val_diease_outputs_accuracy: 0.9376 - val_grow_outputs_accuracy: 0.9578
Epoch 2/1000
236/236 [==============================] - 169s 716ms/step - loss: 0.3876 - diease_outputs_loss: 0.2707 - grow_outputs_loss: 0.1169 - diease_outputs_accuracy: 0.9046 - grow_outputs_accuracy: 0.9600 - val_loss: 0.2467 - val_diease_outputs_loss: 0.1802 - val_grow_outputs_loss: 0.0665 - val_diease_outputs_accuracy: 0.9460 - val_grow_outputs_accuracy: 0.9772
Epoch 3/1000
236/236 [==============================] - 170s 718ms/step - loss: 0.2854 - diease_outputs_loss: 0.2090 - grow_outputs_loss: 0.0764 - diease_outputs_accuracy: 0.9379 - gro